In [ ]:
import os
import pandas as pd

folder_path = "./data/"
save_path = "./"

csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
csv_files.sort()  
df_list = []
for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path, encoding='utf-8') 
    df_list.append(df)

df_merged = pd.concat(df_list, ignore_index=True)

output_csv = os.path.join(save_path, "merged_data.csv")
df_merged.to_csv(output_csv, index=False, encoding='utf-8')

print("合并完成，结果保存在:", output_csv)


In [ ]:
file_path = "merged_data.csv"
df = pd.read_csv(file_path)

duplicate_rows = df[df.duplicated()]

print(duplicate_rows)

df_no_duplicates = df.drop_duplicates()

cleaned_file_path = "merged_data_no_duplicates.csv"  
df_no_duplicates.to_csv(cleaned_file_path, index=False)
